In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #to ignore pandas warning
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *


## Data Processing

### Applying One-Hot Encoding and Shuffling

In [2]:
col_names = ["Class", "a1", "a2", "a3", "a4", "a5", "a6", "ID"]
tr_data = pd.read_csv("../monk+s+problems/monks-1.train", sep =" ",  names = col_names)
test_data = pd.read_csv("../monk+s+problems/monks-1.test", sep =" ",  names = col_names)

tr_data = process_monk_data(tr_data)
test_data = process_monk_data(test_data)

print(tr_data)

     Class        ID  a1_1  a1_2  a1_3  a2_1  a2_2  a2_3  a3_1  a3_2  a4_1  \
0        1   data_42     1     0     0     1     0     0     0     1     0   
1        0  data_157     0     1     0     1     0     0     1     0     0   
2        0  data_143     1     0     0     0     0     1     0     1     0   
3        0   data_93     1     0     0     0     1     0     0     1     0   
4        1  data_197     0     1     0     0     1     0     1     0     1   
..     ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
119      1  data_212     0     1     0     0     1     0     1     0     0   
120      1  data_206     0     1     0     0     1     0     1     0     0   
121      1  data_414     0     0     1     0     0     1     0     1     1   
122      1  data_153     0     1     0     1     0     0     1     0     0   
123      0  data_144     1     0     0     0     0     1     0     1     0   

     a4_2  a4_3  a5_1  a5_2  a5_3  a5_4  a6_1  a6_2  
0       0

## Creating the Neural Network

In [3]:

'''net = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net.add_input_layer(n_inputs)

net.add_hidden_layer(n_inputs, 4)

net.add_output_layer(4, 1)
'''

'net = NeuralNetwork()\n\nn_inputs = tr_data.shape[1] - 2\n\nnet.add_input_layer(n_inputs)\n\nnet.add_hidden_layer(n_inputs, 4)\n\nnet.add_output_layer(4, 1)\n'

## Training

### Training with Mean Square Error

In [4]:
hyperparams = {"epochs": 500,
               "eta": 0.5,
               "clip_value": 1000,
               "hid_act_fun": "tanh",
               "out_act_fun": "sigmoid",
               "cost_fun": "mse",
               "mb" : 16,
               "momentum" : 0.01}


#losses = net.train(tr_data, hyperparams)  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

#plot_loss(losses, cost_fun = hyperparams["cost_fun"])

### Training with Binary Cross Entropy

In [5]:
'''
#RESET THE NETWORK!!!

losses = net.train(tr_data_X, tr_data_Y, epochs=1000,
                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

plot_loss(losses, cost_fun="b_ce")
'''

'\n#RESET THE NETWORK!!!\n\nlosses = net.train(tr_data_X, tr_data_Y, epochs=1000,\n                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE\n# print(net)\n# print(net.input_layer.weights)\n\nplot_loss(losses, cost_fun="b_ce")\n'

# Test Monk 1

In [6]:
#best_eta = net.k_fold(3, tr_data, hyperparams, "eta", [0.01, 0.05, 0.1,0.5, 0.8, 1])
#print(best_eta)

In [7]:
grid = {
    "eta" : [0.8],
    "mb" : [32],
    "momentum" : [ 0.8],
    "n_layers" : [1],
    "n_neurons" : [4],
    "epochs" : [400],
    "clip_value" : [None],
    "hid_act_fun" : ["tanh"],
    "out_act_fun" : ["sigmoid"],
    "cost_fun" : ["b_ce"],
    "ridge_lambda": [None],
    "lasso_lambda": [None],
    "linear_decay": [None],
    "eta0": [None],
    "max_steps": [None],
    "epochs_update": [None]}
    

search_space = get_search_space(grid)

print(len(search_space))
r = []

grid_results = parallel_grid_search(10, tr_data, search_space, n_inputs = tr_data.shape[1] - 2, n_outputs=1)

print(grid_results)
'''
[{'eta': 0.8, 'mb': 32, 'momentum': 0.8, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, array([0.00019234])]
'''


1
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
Valid error:[0.00018338] with par {'eta': 0.8, 'mb': 32, 'momentum': 0.8, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}
shared_res : [{'eta': 0.8, 'mb': 32, 'momentum': 0.8, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, array([0.00018338])], min err: [0.00018338]
GRID SEARCH FINISHED
[{'eta': 0.8, 'mb': 32, 'momentum': 0.8, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cos

"\n[{'eta': 0.8, 'mb': 32, 'momentum': 0.8, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, array([0.00019234])]\n"

In [8]:
np.random.seed(0)
net = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net.add_input_layer(n_inputs)

net.add_hidden_layer(n_inputs, 4)

net.add_output_layer(4, 1)

print(net.hidden_layers[0].weights)
print("net.hidden_layers[0].weight\n")

net5 = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net5.add_input_layer(n_inputs)

net5.add_hidden_layer(n_inputs, 5)

net5.add_output_layer(5, 1)

print(net5.hidden_layers[0].weights)
print(len(net5.hidden_layers))

print("net.hidden_layers[0].weight\n")

net4 = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net4.add_input_layer(n_inputs)

net4.add_hidden_layer(n_inputs, 4)

net4.add_output_layer(4, 1)

print(net4.hidden_layers[0].weights)
print("net.hidden_layers[0].weight\n")

[[-0.27999835  0.0160307   0.22500223  0.03980499]
 [ 0.35162669 -0.38107552  0.12416533 -0.21133698]
 [-0.26873383 -0.17863688 -0.0961472   0.017333  ]
 [-0.35957305  0.27800109  0.14370655 -0.47409509]
 [ 0.4592846   0.5850841   0.36377927 -0.05552601]
 [-0.33044143  0.32541087 -0.12442311  0.37725474]
 [ 0.06427505  0.30139735  0.10997706  0.21805321]
 [ 0.00324038  0.5511316   0.03916592  0.12405661]
 [ 0.58115292 -0.41592748 -0.39208018  0.29916232]
 [-0.36203374  0.59981452 -0.12764559 -0.23066956]
 [ 0.59343279  0.45689678  0.57634121  0.27961145]
 [-0.26578002  0.58945884 -0.08270764  0.24764342]
 [ 0.2923283  -0.04783715  0.18950901  0.28459916]
 [ 0.11616744 -0.33928245  0.0920383   0.40933157]
 [-0.2143483  -0.04617822 -0.1342913   0.57069518]
 [ 0.20747467  0.12574545 -0.23760126  0.16641591]
 [-0.20810358  0.00982312 -0.19622636  0.20875185]]
net.hidden_layers[0].weight

[[ 0.57163209  0.35541541 -0.05424938 -0.32284406  0.31792916]
 [-0.12156242  0.36858106  0.06279727  0